In [7]:
import MySQLdb

db = MySQLdb.connect(  # esto hace la conexion a la base de datos
    host="localhost",  # el puerto es 3308 porque es el puerto que
    user="root",  # le deje a mariaDB
    password="zamromxd",
    database="PokeSearch",
    port=3308,
)
c = db.cursor()

In [44]:
class Pokemon:
    def __init__(self, name: str):
        c.execute(f"select id,atk,satk,def,sdef,total_stat,name from pokemon where name='{name}';")
        data = c.fetchall()
        self.id = data[0][0]
        self.atk= data[0][1]
        self.defe= data[0][3]
        self.satk= data[0][2]
        self.sdef= data[0][4]
        self.total= data[0][5]
        self.name = data[0][6]
        c.execute(f"select id_type from pok_typ where id_pokemon={data[0][0]};")
        T = c.fetchall()
        self.types = []
        self.relations = dict()
        for t_id in T:
            self.types.append(t_id[0])
            if t_id[0] not in self.relations:
                self.relations[t_id[0]] = dict()
            c.execute(f"select relation,id_type_2 from relation_type where id_type_1={t_id[0]};")
            temp = c.fetchall()
            for rt in temp:
                if rt[0] not in self.relations[t_id[0]]:
                    self.relations[t_id[0]][rt[0]] = []
                    self.relations[t_id[0]][rt[0]].append(rt[1])
                else:
                    self.relations[t_id[0]][rt[0]].append(rt[1])
        self.moves = {"physical": dict(), "status": dict(), "special": dict()}
        c.execute(
            f"select M.name,M.id_type,M.damage_class from move as M inner join pok_mov as PM on  PM.id_move=M.id and PM.id_pokemon={self.id};"
        )
        M = c.fetchall()
        for att in M:
            if att[1] not in self.moves[att[2]]:
                self.moves[att[2]][att[1]] = list()
            self.moves[att[2]][att[1]].append(att[0])

        def calcular(self,other:Pokemon):
            puntos = 0
            if self.atk>self.satk:
                pri_atk = "physical"
            elif self.atk<self.satk:
                pri_atk = "special"
            if self.defe > self.sdef:
                pri_def = "physical"
            elif self.defe < self.sdef:
                pri_def = "special"
            if other.atk > other.satk:
                other_atk = "physical"
            elif other.atk < other.satk:
                other_atk = "special"
            if other.defe > other.sdef:
                other_def = "physical"
            elif other.defe < other.sdef:
                other_def = "special"
            
            
            

In [49]:
p = "bulbasaur"
poke = Pokemon(p)
# p = input('Nombre del pokemon: ')
#c.execute(f"select * from pokemon where name='{p}';")
#result = c.fetchall()
pokes = dict()
c.execute(f'select name from pokemon where total_stat>={poke.total-20} and total_stat<={poke.total+20} and id!={poke.id};')
result = c.fetchall()
for r in result:
 pokes[r[0]] = Pokemon(r[0])

In [48]:
poke.moves

{'physical': {1: ['cut',
   'bind',
   'headbutt',
   'tackle',
   'body-slam',
   'take-down',
   'double-edge',
   'strength',
   'rage',
   'bide',
   'skull-bash',
   'false-swipe',
   'return',
   'frustration',
   'facade',
   'secret-power',
   'natural-gift'],
  12: ['vine-whip',
   'razor-leaf',
   'bullet-seed',
   'seed-bomb',
   'power-whip',
   'grassy-glide',
   'trailblaze'],
  16: ['outrage'],
  7: ['fury-cutter'],
  2: ['rock-smash'],
  17: ['knock-off']},
 'status': {1: ['swords-dance',
   'growl',
   'growth',
   'mimic',
   'double-team',
   'defense-curl',
   'flash',
   'substitute',
   'protect',
   'endure',
   'swagger',
   'attract',
   'sleep-talk',
   'safeguard',
   'sweet-scent',
   'nature-power',
   'helping-hand',
   'captivate',
   'work-up',
   'confide'],
  12: ['leech-seed',
   'sleep-powder',
   'synthesis',
   'ingrain',
   'grass-whistle',
   'worry-seed',
   'grassy-terrain'],
  4: ['poison-powder', 'toxic'],
  7: ['string-shot'],
  14: ['light-

In [50]:
poke.relations

{4: {'double_damage_from': [5, 14],
  'double_damage_to': [12, 18],
  'half_damage_from': [2, 4, 7, 12, 18],
  'half_damage_to': [4, 5, 6, 8],
  'no_damage_to': [9]},
 12: {'double_damage_from': [3, 4, 7, 10, 15],
  'double_damage_to': [5, 6, 11],
  'half_damage_from': [5, 11, 12, 13],
  'half_damage_to': [3, 4, 7, 9, 10, 12, 16]}}